In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!nvidia-smi

In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from sklearn.model_selection import train_test_split
import shutil
import csv
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

## Split the folders

In [6]:
def split(data_path,train,validation,split_size=0.2):
    folders = os.listdir(data_path)
    for f in folders:
        fullpath = os.path.join(data_path,f)
        image = glob.glob(os.path.join(fullpath,'*.png'))
       
        x_train,x_val = train_test_split(image,test_size = split_size)
        print("folder name",f,len(x_train))
       
        for x in x_train:
         #   print(x)
            path_to_folder = os.path.join(train,f)
           
            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)
            shutil.copy(x,path_to_folder)
       
        for y in x_val:
            path_to_folder = os.path.join(validation,f)
            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)
            shutil.copy(y,path_to_folder)

In [7]:
data_path = "/kaggle/input/gtsrb-german-traffic-sign/train"
if not os.path.isdir('/kaggle/working/Training'):
    os.mkdir('/kaggle/working/Training')
if not os.path.isdir('/kaggle/working/Validation'):
    os.mkdir('/kaggle/working/Validation')
    
train = '/kaggle/working/Training'
validation = '/kaggle/working/Validation'
## Split train and validation
split(data_path, train, validation, split_size=0.2)

## Prepare Test Data

In [21]:
df = pd.read_csv("/kaggle/input/gtsrb-german-traffic-sign/Test.csv")
df['Path'] = df['Path'].str.replace('Test/', '')
df.to_csv('/kaggle/working/Test1.csv')
os.mkdir("/kaggle/working/Test")

In [27]:
def prepare_test(path_to_image,path_file):

    with open(path_file,"r") as csvfile:
        r= csv.reader(csvfile,delimiter =',')
  
        for i,row in enumerate(r):
            if i == 0: 
                continue
            label = row[-2]
            img_name = row[-1]
            
            dest = os.path.join('/kaggle/working/Test/',label)
            if not os.path.isdir(dest):
                os.makedirs(dest)
            
            to_move = os.path.join(path_to_image,img_name)
            shutil.copy(to_move,dest)

In [28]:
path_to_image = "../input/gtsrb-german-traffic-sign/Test"
path_file ='/kaggle/working/Test1.csv'
prepare_test(path_to_image, path_file)

## Generators: train, validation, test

In [30]:
def create_generators():
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.1, # Shear angle
            zoom_range=0.2, # go from 0.8 to 1.2
            width_shift_range=0.1,
            height_shift_range=0.1,
            rotation_range=10 # Degrees
    )
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
            '/kaggle/working/Training',
            target_size=(32, 32),
            batch_size=32,
            class_mode='categorical')
    validation_generator = validation_datagen.flow_from_directory(
            '/kaggle/working/Validation',
            target_size=(32, 32),
            batch_size=32,
            class_mode='categorical')
    test_generator = test_datagen.flow_from_directory(
            '/kaggle/working/Test',
            target_size=(32, 32),
            batch_size=32,
            class_mode='categorical')
    return (train_generator,validation_generator,test_generator)

In [33]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(30, 30, 3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compilation of the model
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [34]:
model.summary()

In [39]:
# os.mkdir('/kaggle/working/model')
path_to_save_model = '/kaggle/working/model'
chkpt_saver = ModelCheckpoint(
    path_to_save_model, monitor='val_accuracy',
    mode ='max',save_best_only= True,
    save_freq='epoch',verbose = 1
    )

train_generator,validation_generator,test_generator = create_generators()

history = model.fit(train_generator, epochs=50, batch_size=32, 
                    validation_data=validation_generator, callbacks=[chkpt_saver])

In [42]:
model = load_model(path_to_save_model)

In [44]:
print("validation set")
model.evaluate(validation_generator)
print("test set")
model.evaluate(test_generator)

In [45]:
train_1_fnames = os.listdir('/kaggle/working/Training/1')
train_2_fnames = os.listdir('/kaggle/working/Training/2')

print(train_1_fnames[:10])
print(train_2_fnames[:10])

## Visualizing intermediate representations

In [64]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]

visualization_model = Model(inputs = model.input, outputs = successive_outputs)

sign1_files = [os.path.join('/kaggle/working/Training/1', f) for f in train_1_fnames]
sign2_files = [os.path.join('/kaggle/working/Training/2', f) for f in train_2_fnames]

img_path = random.choice(sign1_files + sign2_files)
img = load_img(img_path, target_size=(30, 30))  # this is a PIL image

x   = img_to_array(img)                          
x   = x.reshape((1,) + x.shape)                  

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]


# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=((scale * n_features), scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' )

In [65]:
model.summary()

## Plot the training/validation accuracy

In [66]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

## Quantization aware training

In [70]:
!pip install -q tensorflow-model-optimization

In [71]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

#Compilation of the model
q_aware_model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [72]:
q_aware_model.summary()

In [73]:
q_aware_model.fit(train_generator, epochs=2, batch_size=32, 
                    validation_data=validation_generator)

In [77]:
print("validation set")
q_aware_model.evaluate(validation_generator)
print("test set")
q_aware_model.evaluate(test_generator)

In [78]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_qaware_model = converter.convert()

In [79]:
len(tflite_qaware_model)

## Without quantization

In [80]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
len(tflite_model)

In [82]:
with open("tflite_qaware_model.tflite", 'wb') as f:
    f.write(tflite_qaware_model)